## Box filtering the raw tif images in a (15x15) box.
### Steps:
1. Read metadata for the given experiment and acquire
2. [skip for now] For each file, collect all the unique pixel values (apparently, the number of unique pixels is below 10 for at least one of the experiments)
3. [skip for now] Manually check the distribution of the unique pixel counts, and find a threshold value for the box filter. [Use threshold=79. Can transform this step to an ML algorithm for feature-finding]
4. Box filter all the merged images, and write them into the boxfiltered/ folder.

In [ ]:
# Dependencies
%matplotlib notebook

import xml.etree.ElementTree as ET
import numpy as np
import glob
import cv2
import sys
import time

import colony_iomethods as cm

from matplotlib import pyplot as plt

## File-system organization
- exp_name: The name of the experiment as defined in the microscope computer
- acq_name: Name of the acquire (Typically comes in either an xyz [tilescan + z-stack] OR xyzt [xyz + timepoints])
- base_folder [test_base_folder before publishing]: Currently uses the D: external hardrive at home computer.
- Eventually, will need to distinguish different types of acquisition, such as xyz, xyzt and z-stacks. *[metadata must have this information]*

In [ ]:
# Folder for the files
exp_name = "EQ59_Single_Colony_TilesScan.lif"
# acq_name = "3dTimeScan_17h_init_long"
acq_name = "3dTimeScan_12h_init"

base_folder = f"D:/Tolga/Colony Images/{exp_name}/{acq_name}/"

# Replace the metadata_path with
metadata_path = base_folder + f"MetaData/{exp_name}_{acq_name}_Properties.xml"

tree = ET.parse(metadata_path)    # xml tree of the current stage position
root = tree.getroot()           # root of the xml tree

image_xml = root[0]

# Merged images folder
merged_folder = base_folder + "Merged/"
# Boxfiltered images folder
boxfiltered_folder = base_folder + "Boxfiltered/"

### Test file for obtaining the image dimensions for each tif file
- For each acquisition, the size of the merged tif file should be the same. 
- Must check the {t_str} and {z_str} and {ch_str} before proceed.

In [ ]:
merged_path = merged_folder + f"{acq_name}_t0_z000.tif"
# Image dimension description
dim_desc = cm.collectImageDim(image_xml)

img_test = cv2.imread(merged_path)
(height, width, layers) = img_test.shape

print("Image shape: (h%d,w%d,l%d)" % (height, width, layers))

### Box filter threshold (feature identification):
- At the moment, let's use the threshold pixel value at threshold=80. The value comes from the code at the end of the downsampling notebook
- In the future, this can be reorganized to use an ML algorithm for finding this threshold.

In [ ]:
boxfilter_threshold = 80
# Size of the box
box_size = (15, 15)

## Box filter and write new image to a file
- Depending on the acquisition, xyz, xyzt or z-stack, all of the merged files will be box filtered based on the boxfilter-treshold and written inside the Boxfiltered/ folder

**[Unfinished] Implement a function for each of the acquisition type**

In [ ]:
# For a set number of t
# dim_desc["tsz"] = 1
# For a set number of z
# dim_desc["zsz"] = 1
for tix in range(dim_desc["tsz"]):
    t_str = "t%d" % (tix)
    img_zsum = np.zeros(dim_desc["zsz"])
    for zix in range(dim_desc["zsz"]):
        z_str = "z%03d" % (zix)
        merged_path = merged_folder + f"{acq_name}_{t_str}_{z_str}.tif"
        boxfiltered_path = boxfiltered_folder + f"{acq_name}_{t_str}_{z_str}.tif"
        print(f"Loading file = {merged_path}", end="\r", flush=True)
        
        img = cv2.imread(merged_path)
        
        # Wait for the merge to complete
        while (type(img) == type(None)):
            time.sleep(1) # wait 1 sec and retry
            img = cv2.imread(merged_path)
            
        # Threshold based on the boxfilter_threshold (max set to 1)
        _, thresh = cv2.threshold(img, boxfilter_threshold, 1, cv2.THRESH_BINARY)
        # Boxfilter, accumulate the kernel sized box, and write to the center
        filtered = cv2.boxFilter(thresh, -1, box_size, normalize=False)

        cv2.imwrite(boxfiltered_path, filtered)